In [101]:
import pandas as pd
import numpy as np
import plotly.express as px
from itertools import product

In [102]:
planning = pd.read_excel('../dataset/a6_booster_delivery_planning.xlsx')
tracks = pd.read_excel('../dataset/ariane_tracks_booster_container_2.0.xlsx')

tracks['Entry'] = pd.to_datetime(tracks['Entry'], format='%d.%m.%Y')
tracks['Exit'] = pd.to_datetime(tracks['Exit'], format='%d.%m.%Y')

In [103]:
sorted_tracks = tracks.sort_values(by=['Container','Entry'])

In [104]:
dates = pd.date_range(start=min(sorted_tracks['Entry']), end=max(sorted_tracks['Entry'])).to_list()
containers = list(sorted_tracks['Container'].unique())

In [105]:
df = pd.DataFrame(list(product(dates,containers)), columns=['Date', 'Container'])

In [106]:
crossed_data = (df
                .merge(sorted_tracks[['Container', 'Zone', 'Entry', 'Exit']], how = 'cross')
                .query('Entry <= Date <= Exit')
)

In [107]:
crossed_data = crossed_data[crossed_data['Container_x'] == crossed_data['Container_y']].rename(columns={'Container_x':'Container'}).drop(columns='Container_y').reset_index(drop=True)